In [2]:
import os
import re
import cv2
import json
import base64
from PIL import Image
from io import BytesIO
from typing import Union
import concurrent.futures
from pytube import YouTube
from moviepy.editor import VideoFileClip
from pytube.exceptions import PytubeError

from openai import OpenAI
client = OpenAI(api_key='sk-')

def download_youtube_video(video_url: str) -> str:
    """
    Downloads the highest resolution video from a given YouTube URL.

    Args:
        video_url (str): The URL of the YouTube video to download.

    Returns:
        str: The filename of the downloaded video.

    Raises:
        RuntimeError: If an error occurs during the download.
    """
    try:
        youtube = YouTube(video_url)
        stream = youtube.streams.get_highest_resolution()
        filename = f"{youtube.title}.mp4".replace('/', '_').replace('\\', '_')
        stream.download(filename=filename)
        return filename
    except PytubeError as e:
        raise RuntimeError(
            f"An error occurred while downloading the video: {e}")


def extract_audio_from_video(video_file_path: str) -> str:
    """
    Extracts the audio from a video file and saves it as an MP3.

    Args:
        video_file_path (str): The file path of the video file to extract audio from.

    Returns:
        str: The filename of the saved MP3 audio file.

    Raises:
        RuntimeError: If an error occurs during the audio extraction.
    """
    try:
        video_clip = VideoFileClip(video_file_path)
        audio_clip = video_clip.audio
        head, tail = os.path.split(video_file_path)
        file_name, _ = os.path.splitext(tail)
        mp3_filename = f"{file_name}.mp3"
        audio_clip.write_audiofile(mp3_filename, logger=None)
        audio_clip.close()
        video_clip.close()
        return mp3_filename
    except Exception as e:
        raise RuntimeError(f"An error occurred while extracting audio: {e}")


def extract_frames_from_video(video_path: str, output_folder: str, time_interval: int):
    """
    Extracts individual frames from a video at a specific time interval and provides the paths to these images in ascending order.

    Args:
        - video_path (str): The path to the video file.
        - output_folder (str): The directory where the extracted frames should be saved.
        - time_interval (int): The time interval in seconds at which to extract frames.

    Returns:
        - A list of the saved image file paths.

    Raises:
        - ValueError: If the video cannot be opened.
        - RuntimeError: If an error occurs during frame extraction.
    """
    video_capture = None
    image_paths = []

    try:
        video_capture = cv2.VideoCapture(video_path)
        if not video_capture.isOpened():
            raise ValueError("Could not open video.")

        total_frames = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))
        fps = video_capture.get(cv2.CAP_PROP_FPS)
        duration_seconds = total_frames / fps

        if not os.path.exists(output_folder):
            os.makedirs(output_folder)

        current_time = 0
        while current_time <= duration_seconds:
            frame_number = int(fps * current_time)
            if frame_number >= total_frames:
                break

            video_capture.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
            success, frame = video_capture.read()
            if not success:
                current_time += time_interval
                continue

            output_image_path = os.path.join(
                output_folder, f'{current_time:04d}.jpg')
            cv2.imwrite(output_image_path, frame)
            image_paths.append(output_image_path)
            current_time += time_interval

        return image_paths

    except Exception as e:
        raise RuntimeError(f"An error occurred while extracting frames: {e}")
    finally:
        if video_capture:
            video_capture.release()


def transcribe_audio(audio_file_path: str, segment_len: int = 0, segment_start: int = 0) -> Union[str, list[str]]:
    """
    Transcribes the given audio file and optionally segments the transcription with timestamps.

    Args:
        audio_file_path (str): The file path of the audio file to transcribe.
        segment_len (int): The length of each segment in seconds for timestamping. If 0, no segmentation is done.
        segment_start (int): The start time in seconds for the first segment.

    Returns:
        Union[str, List[str]]: The transcribed text as a string if segment_len is zero, or a list of timestamped segments as strings if segment_len is not zero.
    """
    # Assuming `client` is a pre-defined global variable or imported module with the `audio.transcriptions.create` method.
    with open(audio_file_path, "rb") as audio_file:
        transcript = client.audio.transcriptions.create(
            file=audio_file,
            model="whisper-1",
            response_format="verbose_json",
            timestamp_granularities=["segment"],
        )

    # If segment_len is zero, return the transcript as a string.
    if segment_len == 0:
        return transcript.text

    # Transcript is a Pydantic model, thus converting it to a dictionary
    transcript = transcript.model_dump()

    # Helper function to format timestamps.
    def format_timestamp(seconds: int) -> str:
        hours, remainder = divmod(seconds, 3600)
        minutes, seconds = divmod(remainder, 60)
        return f"{hours:02}:{minutes:02}:{seconds:02}"

    # Initialize variables for processing segments.
    segment_text = ""
    formatted_transcript = []

    # Assuming the transcript is a dictionary with a 'segments' key that contains a list of segment dictionaries.
    segments = transcript.get('segments', [])

    for segment in segments:
        start_time = segment.get('start')
        end_time = segment.get('end')
        text = segment.get('text').strip()

        if segment_text and (start_time - segment_start >= segment_len or segment is segments[-1]):
            formatted_segment = f"{format_timestamp(segment_start)} - {format_timestamp(segment_start + segment_len)} {segment_text}"
            formatted_transcript.append(formatted_segment)
            segment_start += segment_len
            segment_text = text
        else:
            segment_text += " " + text

        while start_time - segment_start >= segment_len:
            segment_start += segment_len

    # Add the last segment if there is any text left.
    if segment_text:
        formatted_segment = f"{format_timestamp(segment_start)} - {format_timestamp(segment_start + (segment_len if segment_start + segment_len <= end_time else end_time))} {segment_text.strip()}"
        formatted_transcript.append(formatted_segment)

    return formatted_transcript


def create_prompt(encoded_images: list[str], prompt_template: str):
    """
    Combines Prompt String with Multiple Images to create message array 
    To be Passed to GPT Vision model

    Parameters:
    - encoded_images:
    - prompt_template:

    Returns:
    - messages:
    """
    messages = [{
        "role": "user",
        "content": [
            {"type": "text", "text": prompt_template}
        ] + [
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{base64_image}",
                    "detail": "high"
                }
            } for base64_image in encoded_images
        ]
    }]
    return messages


def convert_images_to_base64(image_data_list: list[bytes]) -> list[str]:
    """
    Converts a list of image data to a list of Base64 encoded images.

    Parameters:
    - image_data_list: List of bytes objects containing image data.

    Returns:
    - List of Base64 encoded strings representing each image.
    """
    encoded_images = []

    for image_data in image_data_list:
        # Open the image from bytes data
        image = Image.open(BytesIO(image_data))

        # Convert the PIL image to a byte stream
        img_byte_arr = BytesIO()
        image.save(img_byte_arr, format='JPEG')

        # Encode the image to Base64 and convert bytes to string
        encoded_img_str = base64.b64encode(
            img_byte_arr.getvalue()).decode('utf-8')

        encoded_images.append(encoded_img_str)

    return encoded_images


#######################################


def extract_json_from_response(model_response: str) -> str:
    """
    Extract JSON from OpenAI Responce
    """
    PATTERN_JSON_BLOCK = r"```json\n(.*?)\n```"

    json_blocks = re.findall(
        PATTERN_JSON_BLOCK, model_response, re.DOTALL
    )

    try:
        extracted_json = json_blocks[0]
    except IndexError:
        raise ValueError("JSON not found in model response")

    return extracted_json


def format_duration(seconds):
    """
    Formats the duration from seconds to a string of the form "XX hrs XX minutes and XX seconds long".
    If the duration is less than a minute, it returns "XX seconds long".

    :param seconds: The duration of the video in seconds.
    :return: A formatted string representing the duration.
    """
    if seconds >= 3600:  # More than 1 hour
        hours = seconds // 3600
        minutes = (seconds % 3600) // 60
        seconds = seconds % 60
        return f"{hours} hrs {minutes} minutes and {seconds} seconds"
    elif seconds >= 60:  # More than 1 minute
        minutes = seconds // 60
        seconds = seconds % 60
        return f"{minutes} minutes and {seconds} seconds"
    else:  # Less than 1 minute
        return f"{seconds} seconds"


def get_segment_timestamp(segment_string):
    timestamp_pattern = r'\d{2}:\d{2}:\d{2} - \d{2}:\d{2}:\d{2}'
    match = re.search(timestamp_pattern, segment_string)

    if match:
        transcript_timestamp = match.group()
    else:
        transcript_timestamp = ""

    return transcript_timestamp


PROMPT_TEMPLATE = """Your task is to write segments of professional blog/article complete with relevant links, code and Images, based on a YouTube video titled "{video_title}".

Attached are five images sequentially, extracted from **{segment_len}** second segment of a video.  

The images paths are:
```
{image_paths}
```

These images are numbered as the time in seconds from the beginning of the video (dot) jpg. 
The complete video duration is **{video_length}**

Below is the transcript between time: {timestamp} of the video:
```
{transcript_segment}
```

Your final aim is to extract USEFUL INFORMATION in DETAIL from the images that closely relates with the video transcription.
The information that you will extract, will be used to create a professional article/tutorial/blog from the video. 

Based on the images and the transcript, for each image output the following JSON:
{json_template}

**IMPORTANT:**
- Make sure that in the Summary, you should include **ALL** the text and properly formatted **code** from the image
- Note that the `summary` of relevent images will be used to create the final blog.
- Output **JSON**

Take a DEEP breath and Think Step By Step.
"""


JSON_TEMPLATE = """```json
[
    {{
        "image_path": "name/path of the image. Note: The name is the time in seconds at which the frame was captured from the video",
        "summary": "**A DETAILED Summary of the image for the blog**, including relevant Content from the image frame and additional context that relates to the video transcription",
        "relevancy_score": "score between 0 to 100 about relevancy of the image with respect to the transcript",
        "necessity_to_include": "The absolute Necessity of the frame to be included in blog, between 0 to 100, based on the quality of the image and relevance with the transcript context. Only include relevant and extremely IMPORTANT IMAGES"
    }},
    {{
        ...
    }}
    ...
]
```"""



In [3]:
# Number of images to show per transcription

video_url = 'https://youtu.be/3OamzN90kPg?si=TPOz9ny_SoKF35d4'
transcript_segment_length = 20
image_segment_length = 5
concurrent_requests = 25

frames_output_dir = "./extracted_frames"
image_data_dir = "extracted_image_data"

images_per_transcription = transcript_segment_length//image_segment_length


##############

# Download Video
video_file = download_youtube_video(
    video_url=video_url)

# Extract audio from the downloaded video file
extracted_audio = extract_audio_from_video(video_file_path=video_file)

# Extract Frames from video
image_paths = extract_frames_from_video(
    video_path=video_file, output_folder=frames_output_dir, time_interval=4)

# Transcribe audio
transcription = transcribe_audio(
    audio_file_path=extracted_audio, segment_len=transcript_segment_length)


############

# Get formatted length of the video
with VideoFileClip(video_file) as video:
    duration = int(round(video.duration))  # Convert to nearest whole number
    formatted_video_duration = format_duration(duration)

# Gather all the image paths
image_paths_list = []

for img in os.listdir(frames_output_dir):
    image_paths_list.append(f"{frames_output_dir}/{img}")


############


def process_transcription(index, transcript, img_paths_to_print):
    # Calculate timestamp
    timestamp = get_segment_timestamp(segment_string=transcript)

    # Construct the prompt for the GPT model
    describe_images_prompt = PROMPT_TEMPLATE.format(
        video_title=video_file,
        segment_len=transcript_segment_length,
        video_length=formatted_video_duration,
        json_template=JSON_TEMPLATE,
        image_paths="\n".join(img_paths_to_print),
        timestamp=timestamp,
        transcript_segment=transcript,
    )

    # Read image data and convert to base64
    img_data_list = []
    for img_path in img_paths_to_print:
        with open(img_path, 'rb') as img_file:
            img_data = img_file.read()
            img_data_list.append(img_data)
    
    # Encode images to base64
    encoded_images = convert_images_to_base64(img_data_list)

    # Create the prompt
    prompt = create_prompt(encoded_images=encoded_images, prompt_template=describe_images_prompt)

    # Make the API call
    response = client.chat.completions.create(
        model="gpt-4-vision-preview",
        messages=prompt,
        max_tokens=4096,
    )
    
    # Extracting the model response
    model_response = response.choices[0].message.content
    extracted_json = extract_json_from_response(model_response=model_response)

    # Save the result to a json file
    output_folder = image_data_dir 
    os.makedirs(output_folder, exist_ok=True)  # Ensure the directory exists

    file_path = f"{output_folder}/{index}.txt"
    with open(file_path, 'w') as file:
        file.write(model_response)

    file_path = f"{output_folder}/{index}.json"
    json_data = json.loads(extracted_json)
    with open(file_path, 'w') as json_file:
        json.dump(json_data, json_file, indent=4)

    # Optionally return file path or other relevant data
    return file_path


# Use ThreadPoolExecutor to manage a pool of threads
with concurrent.futures.ThreadPoolExecutor(max_workers=concurrent_requests) as executor:
    futures = []
    
    # Make a shallow copy of image_paths_list to avoid threading issues.
    local_image_paths_list = image_paths_list[:]
    
    for i, transcript in enumerate(transcription):
        # Determine the number of images to print for this transcription
        if i == len(transcription) - 1:
            img_paths_to_print = local_image_paths_list
        else:
            img_paths_to_print = local_image_paths_list[:images_per_transcription]
        
        # Update the image_paths_list by removing the printed images
        local_image_paths_list = local_image_paths_list[len(img_paths_to_print):]
        
        # Submit the task to the executor
        future = executor.submit(process_transcription, i, transcript, img_paths_to_print)
        futures.append(future)

    # Wait for all futures to complete
    for future in concurrent.futures.as_completed(futures):
        # You can check for the result or exception here if needed
        try:
            result = future.result()
            # Log the result or store it if necessary
            print(f'Result saved to: {result}')
        except Exception as exc:
            print(f'Generated an exception: {exc}')

Result saved to: extracted_image_data/19.json
Result saved to: extracted_image_data/20.json
Result saved to: extracted_image_data/0.json
Result saved to: extracted_image_data/3.json
Result saved to: extracted_image_data/16.json
Result saved to: extracted_image_data/15.json
Result saved to: extracted_image_data/11.json
Result saved to: extracted_image_data/17.json
Result saved to: extracted_image_data/12.json
Generated an exception: Expecting ',' delimiter: line 10 column 363 (char 1130)
Result saved to: extracted_image_data/5.json
Result saved to: extracted_image_data/8.json
Result saved to: extracted_image_data/1.json
Result saved to: extracted_image_data/14.json
Result saved to: extracted_image_data/9.json
Result saved to: extracted_image_data/10.json
Result saved to: extracted_image_data/2.json
Result saved to: extracted_image_data/13.json
Result saved to: extracted_image_data/7.json
Result saved to: extracted_image_data/18.json
Result saved to: extracted_image_data/6.json


In [7]:
def count_files(directory):
    json_count = 0
    txt_count = 0

    for file_name in os.listdir(directory):
        if file_name.endswith('.json'):
            json_count += 1
        elif file_name.endswith('.txt'):
            txt_count += 1

    return json_count, txt_count

directory_path = "./extracted_image_data"

json_count, txt_count = count_files(directory_path)

print(f"Number of .json files: {json_count}")
print(f"Number of .txt files: {txt_count}")
print(f"Number of Transcription Segments: {len(transcription)}")

Number of .json files: 21
Number of .txt files: 21
Number of Transcription Segments: 21


In [8]:
file_path = "transcription.txt"
with open(file_path, 'w') as file:
    file.write("\n".join(transcription))

print(transcription)

["00:00:00 - 00:00:20  Hey everyone, welcome back, and let's write some more neat code today. So today let's solve contains duplicate. This is another problem from the blind 75 list of questions we've been working on. So I like this problem because it's a good problem for beginners, but there's also multiple solutions to it that I'd like to go over in this video. So we're given an array of numbers.", "00:00:20 - 00:00:40 We want to return true if there's any value in that list of numbers that appears at least twice. So maybe it could appear three times or four times, right? Just at least twice. And we want to return false if there aren't any values that appear at least twice. Basically what that means is that every value in the array is distinct.", "00:00:40 - 00:01:00 So let's take a look at an example. We have one, two, three, and then we have one again. So of course this has duplicates, right? So we return true. And the easiest way we would be able to detect that is by brute forcing

In [21]:

SYSTEM_PROMPT = """ You write Excellent Articles in markdown based on Youtube Transcripts and description of images from the videos.
The aim generate "companion guides" for various tutorials in a more readable, skimmable, searchable format. 
"""

USER_PROMPT = ''' You will be provided with transcription of segments from the video
paired with JSON that contain
- image_path: example "./extracted_frames/0340.jpg", the name of the image `0340` is the number of seconds from the begining of the video
- summary: summary of the emage
- relevancy_score: rough score of how relevent is the image with respect to transcription
- necessity_to_include: necessity of the image to be included in the blog

Below is the transcription between **{timestamp}**
```
{transcript_str}
```

For the above transcript, below is the description of intermediate frames fro mthe video:
```json
{formatted_json_string}
```


Based on the transcription string and the frame/image description json,

Your task is to write a coprehensive segment of the article (which then be used to write the final article) including all the relevent details such as code snippets, links, images and titles in proper **markdown format**

IMPORTANT 1: Keep in mind that this is just one segment of the entire Article so DO NOT conclude the artcile at the end. It will be combined with other segments to create the final article.

IMPORTANT 2: ONLY INCLUDE Relevent and IMPORTANT IMAGES in the article and EXCLUDE redundant images and information

Take DEEP Breath.
Now write the segment of the article in markdown format:

```markdown

```

'''

In [ ]:
image_data_json = []
for file_name in os.listdir(image_data_dir):
    if file_name.endswith('.json'):
        image_data_json.append(f"{image_data_dir}/{file_name}")

print(len(image_data_dir))
print(len(transcription))

In [18]:

####

image_data_json = []
for file_name in os.listdir(image_data_dir):
    if file_name.endswith('.json'):
        image_data_json.append(f"{image_data_dir}/{file_name}")

#####

index = 0

transcript_str = transcription[index]
json_file_path = image_data_json[index]
timestamp = get_segment_timestamp(segment_string=transcript_str)

article_segments = "./article_segments"
###

with open(json_file_path, 'r') as file:
    img_json_data = json.load(file)
    formatted_json_string = json.dumps(img_json_data, indent=4)

user_prompt = USER_PROMPT.format(timestamp=timestamp, transcript_str=transcript_str, formatted_json_string=formatted_json_string)

completion = client.chat.completions.create(
    model="gpt-4-1106-preview",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_prompt},
    ],
    temperature=0.1,
)

model_response = completion.choices[0].message.content

file_path = f"{article_segments}/{index}.txt"
with open(file_path, 'w') as file:
    file.write(model_response)


 You will be provided with transcription of segments from the video
paired with JSON that contain
- image_path: example "./extracted_frames/0340.jpg", the name of the image `0340` is the number of seconds from the begining of the video
- summary: summary of the emage
- relevancy_score: rough score of how relevent is the image with respect to transcription
- necessity_to_include: necessity of the image to be included in the blog

Below is the transcription between **00:00:00 - 00:00:20**
```
00:00:00 - 00:00:20  Hey everyone, welcome back, and let's write some more neat code today. So today let's solve contains duplicate. This is another problem from the blind 75 list of questions we've been working on. So I like this problem because it's a good problem for beginners, but there's also multiple solutions to it that I'd like to go over in this video. So we're given an array of numbers.
```

For the above transcript, below is the description of intermediate frames fro mthe video:
```json
[

In [26]:

####

image_data_json = []
for file_name in os.listdir(image_data_dir):
    if file_name.endswith('.json'):
        image_data_json.append(f"{image_data_dir}/{file_name}")

#####

index = 0

transcript_str = transcription[index]
json_file_path = image_data_json[index]
timestamp = get_segment_timestamp(segment_string=transcript_str)

article_segments = "./article_segments"
###

max_workers = 30

from concurrent.futures import ThreadPoolExecutor, as_completed
from itertools import zip_longest

####

def process_segment(index, transcript_str, json_file_path):
    # If json_file_path is None, we assume there's no corresponding JSON file
    if json_file_path:
        with open(json_file_path, 'r') as file:
            img_json_data = json.load(file)
            formatted_json_string = json.dumps(img_json_data, indent=4)
    else:
        formatted_json_string = "{}"  # Empty JSON representation

    timestamp = get_segment_timestamp(segment_string=transcript_str)
    user_prompt = USER_PROMPT.format(timestamp=timestamp, transcript_str=transcript_str, formatted_json_string=formatted_json_string)

    completion = client.chat.completions.create(
        model="gpt-4-1106-preview",
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": user_prompt},
        ],
        temperature=0.1,
    )

    model_response = completion.choices[0].message.content

    os.makedirs(article_segments, exist_ok=True)  # Ensure the directory exists
    file_path = f"{article_segments}/{index}.txt"
    with open(file_path, 'w') as file:
        file.write(model_response)
    
    return file_path


# Use ThreadPoolExecutor to manage a pool of threads
with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = []

    # Iterate over the transcriptions and image data json paths
    for index, (transcript_str, json_file_path) in enumerate(zip_longest(transcription, image_data_json, fillvalue=None)):
        # Submit the task to the executor
        future = executor.submit(process_segment, index, transcript_str, json_file_path)
        futures.append(future)

    # Wait for all futures to complete and handle results or exceptions
    for future in as_completed(futures):
        try:
            result = future.result()
            print(f'Result saved to: {result}')
        except Exception as exc:
            print(f'Generated an exception: {exc}')

Result saved to: ./article_segments/12.txt
Result saved to: ./article_segments/1.txt
Result saved to: ./article_segments/2.txt
Result saved to: ./article_segments/16.txt
Result saved to: ./article_segments/17.txt
Result saved to: ./article_segments/11.txt
Result saved to: ./article_segments/4.txt
Result saved to: ./article_segments/19.txt
Result saved to: ./article_segments/15.txt
Result saved to: ./article_segments/6.txt
Result saved to: ./article_segments/8.txt
Result saved to: ./article_segments/13.txt
Result saved to: ./article_segments/0.txt
Result saved to: ./article_segments/3.txt
Result saved to: ./article_segments/5.txt
Result saved to: ./article_segments/14.txt
Result saved to: ./article_segments/10.txt
Result saved to: ./article_segments/18.txt
Result saved to: ./article_segments/7.txt
Result saved to: ./article_segments/20.txt
Result saved to: ./article_segments/9.txt


In [35]:
def read_files_in_directory(directory_path):
    formatted_string = ""
    for filename in os.listdir(directory_path):
        if filename.endswith(".txt"):
            file_path = os.path.join(directory_path, filename)
            with open(file_path, 'r') as file:
                formatted_string += f"*** Article Segment Number: {filename} ***\n\n"
                formatted_string += file.read() + "\n\n"
    return formatted_string

directory_path = "./article_segments"
formatted_string = read_files_in_directory(directory_path)

system_prompt = """You are an excellent technical writer. Your task is to combine multiple small articles generated from 20 seconds of video each into a single comprehensive professional Article
"""

user_prompt = f"""Below are all the subarticles sequentially generated from 20 Second segments of the video

Article Segments Start Here ->

```
{formatted_string}
```

<- Article Segments Ends Here

Take a DEEP BREATH

Include 
- relevent images
- code snippets
- links

**IMPORTANT**
**THE ARTICLE SHOULD BE AS DETAILED AS POSSIBLE**

Now use ALL the article segments to write a SINGLE comprehensive and Detailed article COMBINING ALL the article segments in markdwon format:

"""


completion = client.chat.completions.create(
    model="gpt-4-1106-preview",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ],
    temperature=0.85,
)

model_response = completion.choices[0].message.content

print(model_response)

file_path = "final_article.md"
with open(file_path, 'w') as file:
    file.write(model_response)


# Efficient Strategies for Solving the "Contains Duplicate" Problem in Arrays

In the world of coding interviews and algorithm challenges, one frequently encountered question is the "Contains Duplicate" problem, which requires checking for the presence of duplicate values within an array. This task, while seemingly simple, opens up a discussion about various solutions and their associated time and space complexities. In this comprehensive guide, we will dissect multiple strategies for efficiently solving this problem, often highlighted in the "Blind 75" list of coding interview questions.

## The "Contains Duplicate" Challenge

LeetCode's Problem 217, "Contains Duplicate," asks us to determine if any value appears at least twice in a given integer array `nums`. If a duplicate is present, the function should return `true`; otherwise, it should return `false`. This problem is an excellent test of basic data structure knowledge and algorithmic efficiency.

![LeetCode Problem 217: Contains